<a href="https://colab.research.google.com/github/ericsdata/colinsbeer/blob/main/src/BeerSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generating New Reivews

Using a selection of beer revies from the Beer Ratings dataset, we are going to try out generate new text reviews on beer and / or brewer cues.  We will try out several language models - including GPT 2 for text generation. 

The file `write_txt_train.py` has detailed information in how the training set was compiled. Overall, beers that scored at a 4.0 or higher were considered "good".

Resource : https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=U_XJVIetKN-h

In [2]:
## Environment will require HF transformers package
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.1 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 37.2 MB/s 
     |████████████████████████████████| 895 kB 46.7 MB/s 
     |████████████████████████████████| 3.3 MB 34.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Data read in

Working in this colab environment requires a manual upload of the dataset. As google will remind you, this upload expires at the end of the session, so must be reuploaded each time the user runs this notebook. 

Dataset contains beers with at least 30 reviews 

In [3]:
import os
import pandas as pd
## Csv produced by write_txt_train.py file
dat = pd.read_csv(r'..\txt_train.csv')
dat.head(10)

,review_text,good_score
0,"UPDATED: MAR 6, 2004 Amber, with a wonderful o...",0.0
1,"I did not really like this beer. Grainy, bitt...",0.0
2,I was quite pleased with this beer. I tried a ...,0.0
3,Tried UK 3% version in 440ml can. Light in co...,0.0
4,Clean on the palate with a musty biscuity malt...,0.0
5,"Shenandoah Throwdown, growler, thanks wickedpe...",0.0
6,Short stubby bottle from Morrissons clearance ...,0.0
7,Straw color with a tall waxy looking white hea...,0.0
8,Notes of caramel sticky buns on the nose but v...,0.0
9,"A passable lager... meaning, Ill probably pass...",0.0


### Model Loading 

We are relying on HF models to deploy this. 

In [2]:
from transformers import GPT2Tokenizer, GPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

Downloading: 100%|██████████| 0.99M/0.99M [00:01<00:00, 810kB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 1.16MB/s]
Downloading: 100%|██████████| 1.29M/1.29M [00:01<00:00, 1.22MB/s]
Downloading: 100%|██████████| 665/665 [00:00<00:00, 332kB/s]


ImportError: 
GPT2Model requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.


In [1]:
### FAKE DATA

revs = ["On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas or oranges and all around citric. Tastes of oranges, light caramel and a very light grapefruit finish. I too would not believe the 80+ IBUs - I found this one to have a very light bitterness with a medium sweetness to it. Light lacing left on the glass.",
"On tap at the John Harvards in Springfield PA.  Pours a ruby red amber with a medium off whie creamy head that left light lacing.  Aroma of orange and various other citrus.  A little light for what I was expecting from this beers aroma...expecting more from the Simcoe.  Flavor of pine, orange, grapefruit and some malt balance.  Very light bitterness for the 80+ IBUs they said this one had.",
"UPDATED: FEB 19, 2003 Springfield, PA. I've never had the Budvar Cristal but this is exactly what I imagined it to be.  A clean and refreshing, hoppy beer, med bodied with plenty of flavor.  This beer's only downfall is an unpleasant bitterness in the aftertaste.",
"On tap the Springfield PA location billed as the Fancy Lawnmower Light.  Pours a translucent clear yellow with a small bubbly white head.  Aroma was lightly sweet and malty, really no hop presence.  Flavor was light, grainy, grassy and malty.  Just really light in flavor and aroma overall. Watery.",
"On tap at the Springfield, PA location. Poured a lighter golden color with a very small, if any head. Aromas and tastes of grain, very lightly fruity with a light grassy finish. Lively yet thin and watery body. Oh yeah, the person seating me told me this was a new one and was a Pale Ale even though the menu he gave me listed it as a lighter beer brewed in the Kolsh style.",
"Springfield, PA location... Poured an opaque black color with a creamy tan head and nice lacing.  Strong vanilla and roasted malt aroma.  Creamy taste of coffee, chocolate and vanilla. The bartender told me this was an imperial stout at about 8%.  She didn't convince me, there was no alcohol to be found, and it was sweet as hell!  But still good.",
"On tap at the Springfield, PA location. Listed on the beer menu as ""James Brown Ale"". Had the regular and cask version. Poured a deep brown color with an averaged sized off white head (cask had a huge head). Ill stop on the cask version here as I found it to smell and taste like buttered popcorn. The regular had aromas of nuts, light chocolate, and roast. Taste of chocolate, nuts, very light roast and caramel.	 Tasted on 9/7/2006 and moved over as part of the John Harvard clean up."","
"Sampled @ the Springfield, PA location.   Candi Sugar dominates this Belgian Ale.  Beer was on the flat side but had a nice crimson color.   Enjoyable Belgian Ale, I did not expect John Harvards to have it in its line-up.",
"Springfield... Poured a hazy copper color with a medium sized, off white head that left spotty lacing on the glass.  Aroma of yeast, dried fruits, clove, banana, and cherries, with light roastiness.  Aroma was very dubbelish.  Herbal taste of dark fruits, yeast and alcohol was barely noticed.  Slick mouthfeel.  Could have been more flavorful.",
"UPDATED: FEB 19, 2003 Springfield, PA. Darkish copper colored, with no head -	probably poured like that on purpose.	Served inappropriately at about 40 deg	F.  This beer was cold.  It tasted	fine at that temp but I had to let it warm up 	for awhile.  It was worth the wait, as a	very interesting and complex character developed.	Very phenolic and funky - with a strong ester of	bubblegum.  Also a little clove or some kind of 	spice.  Strong but not overwhelming at all.  Surpisingly	easy to drink."","
"UPDATED: FEB 19, 2003 Springfield, PA. Sharp and cloyingly sweet.  The alcohol presence becomes more and more noticeable.",
"UPDATED: FEB 19, 2003 Springfield, PA. Interesting example.  The fruit flavors are very apparent, but the natural mildness of the currants keep the sweetness in check.  These flavors blend well with the white beer base.",
"From Springfield PA:  nice smooth	malty flavor, mildy fruity, but served	via nitro and thru a restrictor disc	(stout tap).  Thus, overly creamy and lacking	some of its original flavor.  I could tell there	was a pretty good beer in there.  Aroma difficult	to detect."","
"On tap at Springfield location.  Pours a translucent golden amber with really no head.  Aroma of caramel, grains and light hops.  Flavor was malty with a light hop presence.  Really kind of non-descript overall.",
"On tap at the Springfield, PA location. Poured a medium and see through orange color with a small sized off white head. Aromas and tastes on the weak side and contained some citrus, caramel, and grains. Body was thin and watery.",
"Handbottled from trade wth Sprinkle. Pours a nice dark copper color with medium size off white head. Aroma of bourbon, malt , hops and oak. Slight smokey flavor with a bourbon taste in the initial sip. Flavors of malt, vanilla and hops still remain although none dominate the brew. Taste is still very enjoyable with a smooth and balanced finish.",
"On tap at the Great Taste of the Midwest (8/9/08): Pours a transparent bright copper orange with an airy white head.  Aroma of sweet toasty pale malt and sweet light fruitiness with a good resiny piny hop character.  Body starts with decent fullness and sweet caramel malts with good balance of hop flavor and bitterness.  Finishes smooth and bittersweet, nicely aged and balanced.",
"UPDATED: JUL 7, 2009 On tap. Interesting experiment, but I liked regular Hopula better. Pours a dark amber with an off white head. Aroma is slightly wood and hops, but mostly bourbon. Flavor is everything I loved about Hopula, but with too much booze. Some nice vanilla notes, but bourbon constantly overpowers the Hopula. Just not my style, but would be very interesting if they reused the barrels.",
"On cask at BI - Aroma of the Hopula Play-Doh hops and malt with lots of oak, vanilla and bourbon.  Pours dark mahogany with a medium lasting head and great lacing.  Flavor is strong bourbon, too strong.  The base beer is hidden under there somewhere but is way overpowered.  I had trouble getting it down to be honest.",
"GTMW 08 on cask - Pours bronze orang with a minimal head.  The aroma has lots of vanilla and some bourbon and toasty malt.  Medium sticky body with light carbonation.  The flavor starts with the aroma traits with more caramel malt and earthy hops.  The finish has vanilla and okay dominating everything.  This kind of beer is just not down my alley."

    
]

In [5]:
import numpy as np
import random
import math
## Set the seed
random.seed(1144)
## Set size of our population
population = [i for i in dat.index]
## Set proportionate size of training set
train_proportion = .7
train_size = math.floor(len(population) * train_proportion)
## Sample the index at the training size ot get the list of index positions for the training set
train_idx = random.sample(population, train_size)

## Make training and testing sets
train =  dat.loc[train_idx]
val = dat.loc[~dat.index.isin(train_idx)]




#####
## This is not used, but a good way to set up data
mldat = {"train" : dat.loc[train_idx].to_dict(orient = "records"),
        "test" : dat.loc[~dat.index.isin(train_idx)].to_dict(orient = "records")}


I define the generative Beer Data. The class takes a text list, adds beginning / ending tags to each element, then tokenizes. 
It returns input_id and attention mask layers

In [ ]:
import torch

class generative_BD(Dataset):
  '''Class for generative text tasks
      This means it adds tags to start and end of texts

      Reads in text dataset, & tokenizes
  '''

  def __init__(self,text_list, tokenizer, gpt2_type="gpt2", max_length=768,text_tags = ['CLS','SEP']):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    ## In definitoin
    for txt in txt_list:

        encodings_dict = tokenizer('<%s>'%(text_tags[0])+ txt + '<%s>'%(text_tags[1]), truncation=True, max_length=max_length, padding="max_length")

        self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
        self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]


class Beer_Data(torch.utils.data.Dataset):
  '''
  Class is a torch data set, each record has encodings and labels
  '''
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

We will used a pretrained distilbert model for this task, fine tuning it for binary classification

In [6]:
train = revs
dataset = generative_BD(train)


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

labels = [0,1]

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset             
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()
trainer.save_model("beer_review_model")

***** Running training *****
  Num examples = 4900
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1839


Step,Training Loss
500,0.033400
1000,0.007100
1500,0.001300


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to beer_review_model
Configuration saved in beer_review_model/config.json
Model weights saved in beer_review_model/pytorch_model.bin


In [ ]:
val = pd.read_csv(r'txt_test.csv')

val_encodings = tokenizer(val['review_text'].astype(str).to_list(), truncation = True, padding = True)

val_dataset = Beer_Data(val_encodings, val['good_score'].to_list())

predictions = trainer.predict(val_dataset)

***** Running Prediction *****
  Num examples = 3000
  Batch size = 4


In [ ]:
test_results=val.copy(deep=True)
test_results["label_int_pred_transfer_learning"]=predictions.label_ids
test_results
#test_results['label_pred_transfer_learning']=test_results['label_int_pred_transfer_learning'].apply(lambda x:labels[x])

#test_results[test_results["good_score"]!=test_results["label_pred_transfer_learning"]].head()

,review_text,good_score,label_int_pred_transfer_learning
0,"Well, tipically bavaria, cheap and fast, not m...",0.0,0.0
1,Packaging sure made this one look deceptively ...,0.0,0.0
2,"Bubbly white head, golden see through. Malty,...",1.0,1.0
3,Bottle pours a light yellow with a thin white ...,0.0,0.0
4,Bottle. Dark ruby colour with a tan head. Arom...,0.0,0.0
...,...,...,...
2995,750ml bottle. Poured with a huge rocky white f...,0.0,0.0
2996,Growler at Shenadoah Throwdown thanks to wicke...,0.0,0.0
2997,"This beer could be a hit with a ""whip out your...",1.0,1.0
2998,Caramel and bitter woody scent. Deep ruby colo...,0.0,0.0


In [ ]:
pd.crosstab(test_results['good_score'], test_results['label_int_pred_transfer_learning'])

label_int_pred_transfer_learning,0.0,1.0
good_score,,
0.0,2891,0
1.0,0,109


In [4]:
dat['good_score'].value_counts()

0.0    6739
1.0     261
Name: good_score, dtype: int64

In [ ]:
max_len = 512
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

test_seq = mldat['train'][1]['review_text']
test_seq2 = mldat['train'][2]['review_text']

seqs = [mldat['train'][1]['review_text'],mldat['train'][2]['review_text'] ]
tokens = tokenizer(seqs, padding = True, 
                            truncation = True, return_tensors = 'pt')
print(tokens)


ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])

model(**tokens)

{'input_ids': tensor([[  101,   101, 26071,  1012, 10364,  2015,  1037,  5024,  2422,  3756,
          2007,  4317,  2132,  1998,  2152,  6351,  3370,  1012, 14747,  2066,
          2087,  7935, 18007,  1010,  2021,  2200,  8217,  1012,  2023,  5404,
          2018,  1037,  2844,  3988, 20943,  5510,  1010,  2029,  2059, 23946,
          2000,  1037,  2300,  2044, 10230,  2618,  1012,  2023,  2419,  2000,
          1037,  4326,  5688,  2004,  2009,  2001,  2667,  2000,  2022,  2119,
          1037,  5744,  4392,  3085,  5404,  1998,  2242,  2007,  6805,  1012,
          1996,  5404,  2001,  2036, 19638,  2084,  3517,  1010,  1998,  1996,
          2132,  8314,  2000, 19219,  2015,  1012,  3452,  1037,  4392,  3085,
          5404,  1010,  2074,  2025,  2200, 22249,  1012,   102,   102],
        [  101,   101, 16958,  2200, 23353,  2100,  1998,  2025,  2428,  2172,
         28126,  1010,  1045,  2876,  2102,  2175,  2041,  1997,  2026,  2126,
          2000,  2031,  2023,  2153,   102, 

SequenceClassifierOutput([('logits', tensor([[ 3.0460, -2.5669],
                                   [ 3.2350, -2.7345]], grad_fn=<AddmmBackward0>))])

In [ ]:
sents = [d['review_text'] for d in mldat["train"]]
tokenized_reviews = tokenizer(sents, padding = True, 
                            truncation = True, return_tensors = 'pt')



len(tokenized_reviews['input_ids'])

4900

In [ ]:
train_InputExamples

4181    InputExample(guid=None, text_a='[CLS]  [SEP]',...
3943    InputExample(guid=None, text_a='[CLS] Bottled....
4025    InputExample(guid=None, text_a='[CLS] Tastes v...
3991    InputExample(guid=None, text_a='[CLS] Bottle: ...
1436    InputExample(guid=None, text_a='[CLS] Pours cl...
                              ...                        
2846    InputExample(guid=None, text_a='[CLS] Bottle p...
6292    InputExample(guid=None, text_a='[CLS] Can. Pou...
1719    InputExample(guid=None, text_a='[CLS] UPDATED:...
1571    InputExample(guid=None, text_a='[CLS] Bottle, ...
1208    InputExample(guid=None, text_a='[CLS] UPDATED:...
Length: 4900, dtype: object

In [ ]:
from transformers import InputExample, InputFeatures

train =  dat.loc[train_idx]
test = dat.loc[~dat.index.isin(train_idx)]

def convert_data_to_examples(train, test, review, sentiment): 
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[review], 
                                                          label = x[sentiment]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[review], 
                                                          label = x[sentiment]), axis = 1,)
  
    return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(train,  test, 'review_text',  'good_score')


In [ ]:



 ## Import BERT Model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

## Moddel inputs holds two arrays
    ## input_ids: list of id of word vector
    ## attention_mask
    
n_batch = 10

batch = sequence [:n_batch]
batch_labs = labels [:n_batch]

model_inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
#model_inputs = tokenizer(sequence, padding = True, truncation = True, return_tensors= 'pt')


## List of IDs
ids = tokenizer.convert_tokens_to_ids(model_inputs)


#batch = tokenizer(few_reviews, padding="max_length", truncation=True, return_tensors="pt")

## !!! Attaching class to model inputs
model_inputs['labels'] = torch.tensor(batch_labs)

'''
### Choosing tokenizer
####    A) Keep reviews by uid
####    B) Sentence strings associated with a particular style

output = model(**model_inputs)

i = 0

for out in range(0,len(output)):

    print(output[i])
    i+=1
'''

'\n### Choosing tokenizer\n####    A) Keep reviews by uid\n####    B) Sentence strings associated with a particular style\n\noutput = model(**model_inputs)\n\ni = 0\n\nfor out in range(0,len(output)):\n\n    print(output[i])\n    i+=1\n'

In [ ]:

optimizer = AdamW(model.parameters())
loss = model(**model_inputs).loss
loss.backward()
optimizer.step()